In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from erddapy import ERDDAP
from erddapy.doc_helpers import show_iframe
from erddapy import servers


import cartopy
cartopy.config['data_dir'] = './maps'
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import cartopy.feature as feat

## Info on the ERDDAP servers recorded by erddapy

In [3]:
#print({k: [v.url,v.description] for k, v in servers.items()}) # dictionary version
print(servers.keys())
print(servers.get('cswc').url)
print(servers.get('ngdac').description)


dict_keys(['cswc', 'apdrc', 'ncei', 'bcodmo', 'emodnet', 'mii', 'cscgom', 'ioos-sensors', 'cencoos', 'cencoos erddap', 'neracoos', 'ngdac', 'pacioos', 'sccoos', 'secoora', 'osmc', 'onc', 'pifsc', 'ooi', 'ooi goldcopy', 'otn', 'myroms', 'rutgers', 'nefsc', 'coops-nos', 'gcoo5-tamu', 'gcoo4-tamu', 'glerl', 'ucsd', 'ubc', 'bmlsc', 'uaf', 'mda', 'ifremer', 'pmel', 'alamo', 'socat', 'hakai', 'nanoos', 'polarwatch', 'nci', 'usgs', 'incois', 'ichec', 'smartatlantic', 'griidc', 'atn-ioos', 'diver', 'jeopp', 'gcoos', 'ceoe', 'cioos atlantic', 'cioos pacific', 'emso eric', 'so-chic', 'ncco', 'canwin'])
https://coastwatch.pfeg.noaa.gov/erddap/
NOAA IOOS NGDAC (National Glider Data Assembly Center)


In [ ]:
e = ERDDAP(
        server='https://nrt.cmems-du.eu/erddap/',
        protocol='tabledap',
    )


In [ ]:
servername= "EMODnet" 
eall = ERDDAP(server=servername)
print(eall.get_search_url(search_for="all",response="html"))
show_iframe(eall.get_search_url(search_for="all",response="html"))
print()
datasetid = eall.get_search_url(search_for="all",response="csv")
df = pd.read_csv(datasetid)
print(
    f'{servername} has {len(set(df["tabledap"].dropna()))} '
    f'tabledap, {len(set(df["griddap"].dropna()))} '
    f'griddap, and {len(set(df["wms"].dropna()))} wms endpoints.'
)
print(df.columns)
#print(df[["Dataset ID","Title"]])

df.loc[df["Title"].str.contains("Temp"),]
#show_iframe(e.get_search_url(search_for="all",response="html"))

In [ ]:
# List all names/institutions/keywords available IN THE SERVER
url = e.get_categorize_url(
    categorize_by="variableName",
    response="csv"
)
print(pd.read_csv(url)["Category"])

In [ ]:
# Refining search: temperature from North Atlantic in 2015

In [6]:
# Constraints
datemin="2015-01-01T00:00:00Z"
datemax="2015-12-31T23:59:59Z"
latmin=15.0
latmax=60.
lonmin=-60.
lonmax=10.
varstandname="sea_water_temperature"

In [ ]:
e = ERDDAP(server="EMODnet", protocol='tabledap')
#show_iframe(e.get_search_url(response="html", **const))
search_url = e.get_search_url(response="csv", **const)
search = pd.read_csv(search_url)
print(search.columns)
print(search[["Title","Dataset ID"]].values)

In [ ]:
e1=e
e1.dataset_id=df.loc[194,"Dataset ID"]
print(e1.dataset_id)

# List of variables
info_url = e.get_info_url(response='csv')
info = pd.read_csv(info_url)
print(info.head(100))
print(info.columns)
print()
#print(info["Attribute Name"].values)

print()

print(" ".join(info.loc[info["Row Type"] == "variable","Variable Name"]))

e1.response="csv"
e1.constraints= {
    "latitude<=": latmax,
    "latitude>=": latmin,
    "longitude>=": lonmin,
    "longitude<=": lonmax,
    "time>=": datemin,
    "time<=": datemax,
}
#e1.get_var_by_attr(dataset_id=e1.dataset_id,
#    standard_name="sea_water_temperature")


df_emodnet = e1.to_pandas()
df_emodnet.columns


In [ ]:
print(df_emodnet.shape)
plt.figure(figsize=(12,12))
ax = plt.axes()
df_emodnet.plot(x='time (UTC)', y='TEMP (degree_Celsius)',ax=ax, 
                linestyle='None', marker='o',grid='on')
plt.show()

In [ ]:
## From the whole server (EMODnet)

In [7]:

servconst = {
   # "standard_name=": varstandname,
    "min_lon=": lonmin,
    "max_lon=": lonmax,
    "min_lat=": latmin,
    "max_lat=": latmax,
    "min_time=": datemin,
    "max_time=": datemax,
    #"cdm_data_type": "trajectoryprofile"
}


In [ ]:
plt.figure(figsize=(12,12))
ax = plt.axes()
df_emodnet2.plot(x='time (UTC)', y='TEMP (degree_Celsius)',ax=ax, 
                linestyle='None', marker='o',grid='on', legend=True)
plt.show()

In [ ]:
show_iframe(e.get_search_url(response="html", **const))


In [ ]:
em = ERDDAP(server="EMSO ERIC", protocol='tabledap')
search_url = em.get_search_url(response="csv", **servconst)
search = pd.read_csv(search_url)
print(search.columns)
print(search[["Title","Dataset ID"]].values)

# Get data
dtype=object
df_emso = em.to_pandas()
print(df_emso.columns)


In [8]:
i = ERDDAP(server="IFREMER", protocol='tabledap')
i.constraints=servconst
search_url = i.get_search_url(response="csv", **servconst)
search = pd.read_csv(search_url)
print(search[["Title","Dataset ID"]].values)

i.dataset_id="ArgoFloats"
i.get_var_by_attr(
    standard_name="sea_water_temperature"
)

df_ifremer = i.to_pandas()
print(df_ifremer.columns)

[['* The List of All Active Datasets in this ERDDAP *' 'allDatasets']
 ['Argo Float Measurements' 'ArgoFloats']
 ['Argo float synthetic vertical profiles : BGC data'
  'ArgoFloats-synthetic-BGC']
 ['Argo Reference Measurements' 'ArgoFloats-ref']
 ['Argo, Mean structure of the North Atlantic subtropical permanent pycnocline'
  'OACP-Argo']
 ['Argo, Profile Classification Model, North-Atlantic, Temperature'
  'PCM-Argo']
 ['ArgoFloats index' 'ArgoFloats-index']
 ['Bioice Station list' 'BIOICE_station_list']
 ['DBCP drifting buoy data and metadata (DBCP drifting buoys GDAC)'
  'DBCPDriftingBuoysGDAC']
 ['Global Ocean, In Situ Observation Copernicus (Copernicus Fishing Observing System)'
  'copernicus-fos']
 ['OceanGliders GDAC trajectories' 'OceanGlidersGDACTrajectories']
 ['SDC Baltic Sea Aggregation V2' 'SDC_BAL_AGG_V2']
 ['SDC Black Sea Aggregation V2' 'SDC_BLS_AGG_V2']
 ['SDC Global Ocean Aggregation V2' 'SDC_GLO_AGG_V2']
 ['SDC Mediterranean Sea Aggregation V2' 'SDC_MED_AGG_V2']
 ['S

HTTPError: Error {
    code=400;
    message="Bad Request: Query error: Unrecognized constraint variable=\"min_lon\".";
}


In [ ]:
plt.figure(figsize=(12,12))
ax = plt.axes()
df_emodnet2.plot(x='time (UTC)', y='TEMP (degree_Celsius)',ax=ax, 
                linestyle='None', marker='o',grid='on', legend=True)
plt.show()

In [ ]:
proj=ccrs.cartopy.crs.Miller()
plt.figure(dpi=150)
ax = plt.axes(projection=proj)
ax.set_extent([lonmin, lonmax, latmax, latmin])
ax.stock_img()
ax.coastlines()
sc=ax.scatter(df_emodnet2['longitude (degrees_east)'],df_emodnet2['latitude (degrees_north)'],
              c=df_emodnet2['TEMP (degree_Celsius)'],s=0.2,
             transform=ccrs.PlateCarree()) # this last one is what projects the data points properly!

plt.colorbar(sc)
plt.show()

In [ ]:



df_emodnetsurf=df_emodnet2[df_emodnet2["depth (m)"]<=-50.]
print(df_emodnetsurf.shape)

# All global attributes in dataset
info_url = e.get_info_url(dataset_id=df.loc[2,"Dataset ID"], response='csv')
info = pd.read_csv(info_url)
info.head()



# search for dataset
#url = e.get_search_url(search_for='v2020', response="csv")
#print(pd.read_csv(url)["Dataset ID"])

# show the list of variables

info_url = e.get_info_url(dataset_id="pmelTaoDyQnet", response='csv')
info = pd.read_csv(info_url)
info.head()


#print(pd.read_csv(datasetid)["Dataset ID"])

dataset_id = "pmelTaoDyQnet"
##Get variables with x-axis attribute.

#e.get_var_by_attr('sd1066_2019')

##['longitude']
##Get variables with matching “standard_name” attribute

#e.get_var_by_attr(
#    dataset_id, standard_name="northward_sea_water_velocity"
#)

In [ ]:
# SOCAT
e = ERDDAP(
  server='https://data.pmel.noaa.gov/socat/erddap',
  protocol='tabledap',
)

e.response = 'csv'
e.dataset_id = 'socat_v2020_fulldata'
e.constraints = {
    'time>=': '2016-07-10T00:00:00Z',
    'time<=': '2019-02-10T00:00:00Z',
#    'latitude>=': -60.0,
#    'latitude<=': 60.0,
#    'longitude>=': -30.0,
#    'longitude<=': 30.0,
    'platform_name=~': "(Station M|Simon Stevin|Polarstern)"
}

e.variables = [
    'platform_name',
    'organization',
    'expocode',
    'time',
    'latitude',
    'longitude',
    'sal',
    'temp',
    'fCO2_water_sst_100humidity_uatm',
    'qc_flag',
    'WOCE_CO2_water'
]

df_socat = e.to_pandas()


In [ ]:
# Argo (Europe)
e = ERDDAP(
  server='https://polarwatch.noaa.gov/erddap/',
  protocol='tabledap',
)
e.response = 'csv'
e.dataset_id = 'SOCCOM_BGC_Argo'
e.constraints = {
    'time>=': '2016-07-10T00:00:00Z',
    'time<=': '2017-02-10T00:00:00Z',
    'region=': 'N. Atlantic'
#    'latitude>=': -60.0,
#    'latitude<=': 60.0,
#    'longitude>=': -30.0,
#    'longitude<=': 30.0,
#    'depth<=': 10,
#    'EP_PLATFORM_CODE=': 'DBBH', #Meteor
}

e.variables = [
    'WMO_ID',
    'time',
    'latitude',
    'longitude',
    'temperature',
    'salinity',
    'depth',
    'pH_insitu',
    'pCO2_LIAR',
    'pCO2_LIAR_QF',
]

df_bgcarco = e.to_pandas()

In [ ]:
# EMODNet Physics 

e = ERDDAP(
  server='https://jeodpp.jrc.ec.europa.eu/services/erddap/',
  protocol='tabledap',
)
e.response = 'csv'
e.dataset_id = 'EP_ERD_INT_TEMP_AL_PR_NRT'
e.constraints = {
    'time>=': '2016-07-10T00:00:00Z',
    'time<=': '2017-02-10T00:00:00Z',
#    'latitude>=': -60.0,
#    'latitude<=': 60.0,
#    'longitude>=': -30.0,
#    'longitude<=': 30.0,
    'EP_PLATFORM_CODE=': 'DBBH', #Meteor
}

e.variables = [
    'time',
    'latitude',
    'longitude',
    'TEMP',
    'depth',
    'TEMP_QC',
]

df_emodnet = e.to_pandas()


In [ ]:
print(df_socat.columns)
print(set(df_socat['platform_name']))
print(np.nanmax(df_socat['fCO2_water_sst_100humidity_uatm (uatm)']))
print(np.nanmin(df_socat['fCO2_water_sst_100humidity_uatm (uatm)']))

print(df_glodapgrid.columns)
print(max(df_glodapgrid['depth_surface (count)']))
df_glodapgrid.shape

print(df_emodnet.columns)

In [ ]:
# GLODAP gridded
ee = ERDDAP(
  server='https://jeodpp.jrc.ec.europa.eu/services/erddap',
  protocol='griddap',
)
ee.response = 'csv'
ee.dataset_id = 'GLODAPv2_2016b_CMEMS'
#ee.constraints = {
#    'latitude>=': -60.0,
#    'latitude<=': 60.0,
#    'longitude>=': -30.0,
#    'longitude<=': 30.0,
#    'depth<=' : 1,
#}
ee.variables = [
    'pHtsinsitutp',
    'temperature',
    'salinity'
]

df_glodapgrid = ee.to_pandas()

In [ ]:

plt.figure(figsize=(12,12))
ax = plt.axes()
df_socat.plot(x='time (UTC)', y='temp (degrees C)',ax=ax,linestyle='None', marker='o',grid='on',legend=True)
df_emodnet.plot(x='time (UTC)', y='TEMP (degree_Celsius)',ax=ax, linestyle='None', marker='o',grid='on')
plt.show()

In [ ]:
plotph=df_glodapgrid.loc[df_glodapgrid['depth_surface (count)'] == 1,:]
#plotph=df_glodapgrid.loc[np.logical_and(df_glodapgrid['depth_surface (count)'] == 1#, 
                        #  df_glodapgrid['pHtsinsitutp'].notnull())
#                         ,:]
print(min(plotph['pHtsinsitutp']))
print(plotph.shape)
print(df_glodapgrid.shape)



print((plotph['latitude (degrees_north)'].unique()).shape)
print((plotph['longitude (degrees_east)'].unique()).shape)

plotph
lat=plotph['latitude (degrees_north)'].unique()
lon=plotph['longitude (degrees_east)'].unique()
phonly=(plotph['pHtsinsitutp']).values.reshape(180,360)
print(plotph)
print(phonly)

In [ ]:
plotph=df_glodapgrid.loc[df_glodapgrid['depth_surface (count)'] == 1 & pd.notnull(df_glodapgrid['depth_surface (count)']),:]

proj=ccrs.cartopy.crs.Miller()
plt.figure(dpi=150)
ax = plt.axes(projection=proj)
#ax.set_extent([-30, 30, 60, -60])
#ax.stock_img()
#land_50m = feat.NaturalEarthFeature('physical', 'land', '50m',edgecolor='grey',facecolor=feat.COLORS['land'])
#ax.add_feature(land_50m)
ax.coastlines()
cont=ax.contourf(lon,lat,phonly,transform=ccrs.PlateCarree())
sc=ax.scatter(df_socat['longitude (degrees_east)'],df_socat['latitude (degrees_north)'],
              c=df_socat['fCO2_water_sst_100humidity_uatm (uatm)'],s=0.1,cmap='Reds',
             transform=ccrs.PlateCarree()) # this last one is what projects the data points properly!
plt.colorbar(sc)

In [ ]:
em = ERDDAP(server="http://erddap.emso.eu/erddap", protocol='tabledap')
search_url = em.get_search_url(response="csv", **servconst)
search = pd.read_csv(search_url)
print(search.columns)
print(search[["Title","Dataset ID"]].values)


em.dataset_id="Emso_Azores_Chemini_IRON"
# List of variables
info_url = em.get_info_url(response='csv')
info = pd.read_csv(info_url)
print(" ".join(info.loc[info["Row Type"] == "variable","Variable Name"]))
print()

em.constraints= {
    "latitude<=": latmax,
    "latitude>=": latmin,
    "longitude>=": lonmin,
    "longitude<=": lonmax,
    "time>=": datemin,
    "time<=": datemax,
    "TEMP_QC=":1,
    "depth<=":10.,
}
em.variables= ['time','latitude','longitude','TEMP']

df_emso = em.to_pandas()
print(df_emso.columns)

In [ ]:
# Maps
proj=ccrs.cartopy.crs.Miller()
plt.figure(dpi=150)
ax = plt.axes(projection=proj)
ax.set_extent([-80, 30, 60, 50])
ax.stock_img()
#land_50m = feat.NaturalEarthFeature('physical', 'land', '50m',edgecolor='grey',facecolor=feat.COLORS['land'])
#ax.add_feature(land_50m)
ax.coastlines()
sc=ax.scatter(df_socat['longitude (degrees_east)'],df_socat['latitude (degrees_north)'],
              c=df_socat['fCO2_water_sst_100humidity_uatm (uatm)'],s=0.2,
             transform=ccrs.PlateCarree()) # this last one is what projects the data points properly!
plt.colorbar(sc)
plt.show()

In [ ]:
import folium

sample_coord= list(zip(df['latitude (degrees_north)'], df['longitude (degrees_east)']))
myMap = folium.Map(location=[40, -70], zoom_start=2)
folium.TileLayer('openstreetmap').add_to(myMap)
folium.TileLayer('stamenterrain').add_to(myMap)
folium.PolyLine(locations = sample_coord, color='black').add_to(myMap)
folium.LayerControl().add_to(myMap)
myMap

# Try something else in folium

myMap=folium.Map([40, -70], zoom_start=8)
#location=location[0:2000]

import branca
import branca.colormap as cm

colormap = cm.LinearColormap(colors=['yellow','blue'], index=[250,600],vmin=250,vmax=600)

loc=zip(df['latitude (degrees_north)'], df['longitude (degrees_east)'])
fco2=df['fCO2_water_sst_100humidity_uatm (uatm)']

folium.ColorLine(
        positions = loc, # tuple of coordinates 
        colors = fco2, # map each segment with the speed 
        colormap =  colormap, # map each value with a color 
        ).add_to(myMap)

#for loc,fco2,temp in zip(zip(df['latitude (degrees_north)'], df['longitude (degrees_east)']), df['fCO2_water_sst_100humidity_uatm (uatm)'], df['temp (degrees C)']):
#     folium.Circle(location=loc,
#                            popup=fco2,
#                            radius=10,
#                            color='blue',#colormap(fco2), # this part doesn't work and I'm not familiar enough with 
#                            fill=True,
#                            fill_opacity=0.7,
#                           ).add_to(myMap)
myMap.add_child(colormap)
myMap

In [ ]:
import plotly.express as px
loc=zip(df['latitude (degrees_north)'], df['longitude (degrees_east)'])

fig = px.scatter_geo(df, lat='latitude (degrees_north)',
                     lon='longitude (degrees_east)',
                     color="fCO2_water_sst_100humidity_uatm (uatm)", # which column to use to set the color of markers
                     hover_name="expocode", # column added to hover information
                   #  size="temp (degrees C)", # size of markers
                     projection="natural earth")
fig.update_geos(fitbounds='locations')

fig.show()

In [ ]:
url='https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0210813/GLODAPv2.2020_Merged_Master_File.csv'
df=pd.read_csv(url, error_bad_lines = False)

In [ ]:
#list(df.columns)
#print(df["G2fco2"].shape)
#print(~pd.isna(df["G2fco2"]).shape)


In [ ]:
surfilt=(df["G2depth"]<=10.) & (~pd.isna(df["G2fco2"])) & (df["G2fco2f"]==2) # Good, MEASURED. Flag 0 is for calculated
dfsurf=df[surfilt] #Only measured fCO2. GLODAP has calculated fco2
dfsurfgood= dfsurf[[
    'G2year','G2month','G2day','G2hour','G2minute',
    'G2cruise','G2station','G2cast',
 'G2latitude',
 'G2longitude',
 'G2depth',
 'G2temperature',
 'G2salinity',
 'G2salinityf',
# 'G2tco2',
# 'G2tco2f',
# 'G2talk',
# 'G2talkf',
 'G2fco2',
 'G2fco2f'
# 'G2fco2temp',
# 'G2phts25p0',
# 'G2phts25p0f',
# 'G2phtsinsitutp',
# 'G2phtsinsitutpf'
]].copy()
dfsurfgood.reset_index(drop=True, inplace=True)

print(df.shape,dfsurf.shape, dfsurfgood.shape)

# Unique cruise/station/cast index
dfsurfgood['UNICAST']=dfsurfgood.set_index(['G2cruise', 'G2station', 'G2cast']).index.factorize()[0]+1

# Change to proper date format
tempdtframe = pd.DataFrame({'year':dfsurfgood['G2year'], 'month':dfsurfgood['G2month'], 'day':dfsurfgood['G2day'],
                       'hour':dfsurfgood['G2hour'], 'minute':dfsurfgood['G2minute'] })
dfsurfgood['DATEVECTOR']=pd.to_datetime(tempdtframe)

# Drop unnecesary columns
dfsurfgood.drop(['G2year','G2month','G2day','G2hour','G2minute',
    'G2cruise','G2station','G2cast'],inplace=True,axis=1)

# Filter for only the uppermost measurement at each unique cast (if, e.g. samples at 2 and 10 m)
surfacemostind=[]
for x in np.unique(dfsurfgood['UNICAST']):
    surfacemostind.append(dfsurfgood['G2depth'].iloc[np.where(dfsurfgood['UNICAST']==x)].idxmin())

glodapfco2surf=dfsurfgood.iloc[surfacemostind].copy()
print(glodapfco2surf)

# Average duplicates (no duplicates for measured CO2, implement later)
glodapfco2surf.shape
len(np.unique(glodapfco2surf['UNICAST']))

In [ ]:
plt.scatter(glodapfco2surf['UNICAST'], glodapfco2surf['G2depth'])
#shorterdf=dumbtemp.iloc[surfacemostind]

In [1]:
from erddapy import ERDDAP
from erddapy.doc_helpers import show_iframe

# Get SOCAT fco2 & temp/sal data
e = ERDDAP(
  server='https://data.pmel.noaa.gov/socat/erddap',
  protocol='tabledap',
)

e.response = 'csv'
e.dataset_id = 'socat_v2020_fulldata'
e.constraints = {
    'dist_to_land>=': 10,
    'time>=': '2016-07-10T00:00:00Z',
    'time<=': '2017-02-10T00:00:00Z',
#    'latitude>=': -60.0,
#    'latitude<=': 60.0,
#    'longitude>=': -30.0,
#    'longitude<=': 30.0,
#    'platform_name=~': "(Station M|Simon Stevin|Polarstern)"
#        'WOCE_CO2_water=': "2" 
            # 'region_id=': A,C,I,N,O,R,T,Z
        # 'expocode=':'74AB19900918',
            # 'fCO2_water_sst_100humidity_uatm=~':"float('nan')" # Have yet to figure out how to set the nan filter

}

e.variables = [
    'expocode',
    'time',
    'latitude',
    'longitude',
    'sal',
    'temp',
    'fCO2_water_sst_100humidity_uatm',
    'qc_flag',
    'WOCE_CO2_water',
    'dist_to_land'
]

df_socat = e.to_pandas()



In [10]:
import pandas as pd
url = e.get_categorize_url(
    categorize_by="variableName",
    response="csv"
)

pd.read_csv(url)["Category"]


0                   all_region_ids
1                         altitude
2                       calc_speed
3                     dataset_name
4                              day
                  ...             
88    xco2_water_equi_temp_wet_ppm
89          xco2_water_sst_dry_ppm
90          xco2_water_sst_wet_ppm
91                       xh2o_equi
92                            year
Name: Category, Length: 93, dtype: object

In [ ]:
#df_socat = e.to_pandas()

df_socat.shape
#print(e)
list(df_socat.columns)

In [2]:
import cartopy
cartopy.config['data_dir'] = './maps'
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import cartopy.feature as feat

proj=ccrs.cartopy.crs.Miller()
plt.figure(dpi=150)
ax = plt.axes(projection=proj)
#ax.set_extent([180, -180, 90, -90])
#ax.stock_img()
ax.coastlines()
sc=ax.scatter(df_socat['longitude (degrees_east)'],df_socat['latitude (degrees_north)'],
              c='orange',s=0.2, label="SOCAT",
             transform=ccrs.PlateCarree())
sc=ax.scatter(glodapfco2surf['G2longitude'],glodapfco2surf['G2latitude'],
              c='blue',s=0.2, label= "GLODAP measured fCO2 surface",
             transform=ccrs.PlateCarree())

#ax.legend()
plt.show()

ImportError: dlopen(/usr/local/lib/python3.9/site-packages/cartopy/_crs.cpython-39-darwin.so, 2): Library not loaded: /usr/local/opt/proj/lib/libproj.19.dylib
  Referenced from: /usr/local/lib/python3.9/site-packages/cartopy/_crs.cpython-39-darwin.so
  Reason: image not found

In [ ]:
min(df_socat['temp (degrees C)'])
np.unique(df_socat['qc_flag']))

In [3]:
cartopy.__version__

NameError: name 'cartopy' is not defined

In [ ]:
socat_g = ERDDAP(
  server='http://erddap.sochic-h2020.eu/erddap',
  protocol='griddap',
        response="opendap",

)
socat_g.dataset_id = 'SOCATv2020_tracks_gridded_yearly'

socat_g.griddap_initialize()

# Can't figure out how to subset by time

#socat_g.constraints['minTime'] = '2000-01-01T00:00:00Z'
#socat_g.constraints['maxTime'] = '2015-12-31T23:59:59Z',
socat_g.griddap_initialize()

socat_g.variables = ['fco2_ave_weighted_year']

df_socatgrid = socat_g.to_xarray()